In [1]:
import pandas as pd
import numpy as np

# Load VR data with all available years/timeframes
vr_data_path = "/Users/borismartinez/Documents/GitHub/engage/data/vr_blocks_export.csv" # update path


vr_df = pd.read_csv(vr_data_path, low_memory=False)
print(f"Total VR records loaded: {len(vr_df)}")

Total VR records loaded: 84616


In [2]:
# List key columns that must not be missing or invalid
key_cols = ["first_name", "last_name"]

# Define values to treat as missing
null_equivs = ['', 'na', 'nan', 'null', 'none', 'undefined']

def is_missing_strict(series):
    # Check nulls plus strings in null_equivs (case-insensitive)
    return series.isna() | series.astype(str).str.strip().str.lower().isin(null_equivs)

missing_counts = {}
for col in key_cols:
    missing_counts[col] = is_missing_strict(vr_df[col]).sum()

missing_df = pd.DataFrame.from_dict(missing_counts, orient='index', columns=['missing_count'])
missing_df['missing_pct'] = (missing_df['missing_count'] / len(vr_df)) * 100
print(missing_df)

usable_mask = ~pd.concat([is_missing_strict(vr_df[col]) for col in key_cols], axis=1).any(axis=1)
vr_df['usable'] = usable_mask

print(f"Total VR records: {len(vr_df)}")
print(f"Usable VR records: {vr_df['usable'].sum()} ({vr_df['usable'].mean()*100:.2f}%)")

# Extract first 10 characters to get the date part only
vr_df['upload_date_str'] = vr_df['upload_time'].astype(str).str[:10]

# Parse the cleaned date strings to datetime
vr_df['upload_date'] = pd.to_datetime(vr_df['upload_date_str'], errors='coerce')

# Check how many are still missing after this cleaning
missing_dates = vr_df['upload_date'].isna().sum()
print(f"Number of records with missing/invalid upload_date after cleaning: {missing_dates}")

# Drop or handle missing those as you prefer
vr_df = vr_df[vr_df['upload_date'].notna()].copy()

# Extract year and month
vr_df['upload_year'] = vr_df['upload_date'].dt.year
vr_df['upload_month'] = vr_df['upload_date'].dt.month

            missing_count  missing_pct
first_name          32432    38.328449
last_name           33023    39.026898
Total VR records: 84616
Usable VR records: 51008 (60.28%)
Number of records with missing/invalid upload_date after cleaning: 0


In [3]:
import numpy as np
florida_counties = [
    'Alachua', 'Baker', 'Bay', 'Bradford', 'Brevard', 'Broward', 'Calhoun', 'Charlotte', 'Citrus',
    'Clay', 'Collier', 'Columbia', 'DeSoto', 'Dixie', 'Duval', 'Escambia', 'Flagler', 'Franklin',
    'Gadsden', 'Gilchrist', 'Glades', 'Gulf', 'Hamilton', 'Hardee', 'Hendry', 'Hernando',
    'Highlands', 'Hillsborough', 'Holmes', 'Indian River', 'Jackson', 'Jefferson', 'Lafayette',
    'Lake', 'Lee', 'Leon', 'Levy', 'Liberty', 'Madison', 'Manatee', 'Marion', 'Martin',
    'Miami-Dade', 'Monroe', 'Nassau', 'Okaloosa', 'Okeechobee', 'Orange', 'Osceola', 'Palm Beach',
    'Pasco', 'Pinellas', 'Polk', 'Putnam', 'Santa Rosa', 'Sarasota', 'Seminole', 'St. Johns',
    'St. Lucie', 'Sumter', 'Suwannee', 'Taylor', 'Union', 'Volusia', 'Wakulla', 'Walton',
    'Washington'
]

# Then you can create the lowercase list
florida_counties_lower = [c.lower() for c in florida_counties]


# List of all 67 Florida counties lowercase for faster membership tests
florida_counties_lower = [c.lower() for c in florida_counties]

# Your misspellings mapping (keys are lowercase misspellings, values are properly capitalized names)
misspellings_map = {
    'dade': 'Miami-Dade',
    'miami dade': 'Miami-Dade',
    'miami-dade': 'Miami-Dade',
    'broward county': 'Broward',
    'palm beach fl': 'Palm Beach',
    'st lucie': 'St. Lucie',
    # add other misspellings you know
}

def clean_county(name):
    if not isinstance(name, str):
        return np.nan
    name_clean = name.strip().lower()
    if name_clean in florida_counties_lower:
        # Already a valid Florida county (lowercase) -> capitalize properly
        idx = florida_counties_lower.index(name_clean)
        return florida_counties[idx]
    elif name_clean in misspellings_map:
        # Known misspelling -> map to proper county name
        return misspellings_map[name_clean]
    else:
        # Unknown or invalid county name
        return np.nan

# Apply function to your raw 'data_entry_county' column
vr_df['county_clean'] = vr_df['data_entry_county'].apply(clean_county)

# Fill missing with 'Missing/Unknown'
vr_df['county_report'] = vr_df['county_clean'].fillna('Missing/Unknown')

In [5]:
from datetime import datetime
import pandas as pd

def calculate_age(dob):
    dob_dt = pd.to_datetime(dob, errors='coerce')
    if pd.isna(dob_dt):
        return None
    today = datetime.today()
    return (today - dob_dt).days // 365

def summarize_vr_coverage(df: pd.DataFrame, year_col: str = None, year_value=None):
    """
    Summarizes VR data coverage statistics overall or filtered by year.

    Args:
        df (pd.DataFrame): VR data with cleaned county and usable columns.
        year_col (str): Name of the column containing year (e.g., 'upload_year').
        year_value: Year value to filter on; if None, no filtering applied (overall).

    Returns:
        dict: Summary DataFrames similar to your existing code.
    """
    if year_col and year_value is not None:
        df = df[df[year_col] == year_value]

    # Ensure age and age_band columns updated if they don't exist
    if 'age' not in df.columns or 'age_band' not in df.columns:
        df = df.copy()
        df['age'] = df['date_of_birth'].apply(calculate_age)
        bins = [0, 24, 34, 49, 150]
        labels = ['18-24', '25-34', '35-49', '50+']
        df['age_band'] = pd.cut(df['age'], bins=bins, labels=labels, right=True)
    else:
        labels = df['age_band'].cat.categories.tolist() if pd.api.types.is_categorical_dtype(df['age_band']) else ['18-24', '25-34', '35-49', '50+']

    # 3. County summary
    county_summary = (
        df.groupby('county_clean')
        .agg(
            total_records=pd.NamedAgg(column='county_clean', aggfunc='size'),
            usable_records=pd.NamedAgg(column='usable', aggfunc='sum')
        )
        .sort_values(by='total_records', ascending=False)
    )

    # 4. Usable records by zip code pivoted
    usable_by_zip = (
        df[df['usable']]
        .groupby(['county_clean', 'voting_zipcode'])
        .size()
        .reset_index(name='count')
    )
    pivot_zip = usable_by_zip.pivot_table(
        index='voting_zipcode', columns='county_clean', values='count', fill_value=0
    )

    # 5. Collection location pivots
    grouping_cols = ['collection_location_zip', 'collection_location_city', 'collection_location_name']
    pivot_collection = {}
    for col in grouping_cols:
        if col in df.columns:
            usable_grouped = (
                df[df['usable']]
                .groupby(['county_clean', col])
                .size()
                .reset_index(name='count')
            )
            pivot_collection[col] = usable_grouped.pivot_table(
                index=col, columns='county_clean', values='count', fill_value=0
            )
        else:
            pivot_collection[col] = pd.DataFrame()

    # Age band pivot
    usable_by_age_band = (
        df[df['usable']]
        .groupby(['county_clean', 'age_band'], observed=True)
        .size()
        .reset_index(name='count')
    )
    pivot_age_band = usable_by_age_band.pivot_table(
        index='age_band',
        columns='county_clean',
        values='count',
        fill_value=0,
        observed=True
    ).reindex(labels)

    # 7. Ethnicity counts
    usable_by_ethnicity = pd.DataFrame()
    if 'ethnicity' in df.columns:
        usable_by_ethnicity = (
            df[df['usable']]
            .groupby(['county_clean', 'ethnicity'])
            .size()
            .reset_index(name='count')
            .sort_values(by=['county_clean', 'ethnicity'])
        )

    result = {
        "county_summary": county_summary,
        "pivot_zip": pivot_zip,
        "pivot_collection": pivot_collection,
        "pivot_age_band": pivot_age_band,
        "usable_by_ethnicity": usable_by_ethnicity
    }
    return result

# ---- USAGE ----

# Convert upload_year to integer with coercion
vr_df['upload_year'] = pd.to_numeric(vr_df['upload_year'], errors='coerce').astype('Int64')

# Overall stats
overall_stats = summarize_vr_coverage(vr_df)

# Per year stats, assuming 'upload_year' column exists
years = vr_df['upload_year'].dropna().unique()
per_year_stats = {}
for y in sorted(years):
    per_year_stats[int(y)] = summarize_vr_coverage(vr_df, year_col='upload_year', year_value=int(y))


In [15]:
years = [2024, 2023, 2022, 2021, 2020, 2019, 2018]

# Overall summaries
print("Overall County Summary:")
print(overall_stats['county_summary'].head())

print("\nOverall Pivot Zip Sample:")
print(overall_stats['pivot_zip'].head())

print("\nOverall Pivot Age Band Sample:")
print(overall_stats['pivot_age_band'].head())

print("\nOverall Ethnicity Summary:")
print(overall_stats['usable_by_ethnicity'].head())

print("\nOverall Collection Location Zip Sample:")
if overall_stats['pivot_collection'].get('collection_location_zip') is not None:
    print(overall_stats['pivot_collection']['collection_location_zip'].head())

print("\nOverall Collection Location City Sample:")
if overall_stats['pivot_collection'].get('collection_location_city') is not None:
    print(overall_stats['pivot_collection']['collection_location_city'].head())

print("\nOverall Collection Location Name Sample:")
if overall_stats['pivot_collection'].get('collection_location_name') is not None:
    print(overall_stats['pivot_collection']['collection_location_name'].head())

# Year-by-year summaries
for year in years:
    print(f"\nCounty Summary for Year: {year}")
    if year in per_year_stats:
        print(per_year_stats[year]['county_summary'].head())
    else:
        print(f"No data available for year {year}")
    
    print(f"\nPivot Zip for Year: {year}")
    if year in per_year_stats:
        print(per_year_stats[year]['pivot_zip'].head())
    else:
        print(f"No data available for year {year}")
    
    print(f"\nPivot Age Band for Year: {year}")
    if year in per_year_stats:
        print(per_year_stats[year]['pivot_age_band'].head())
    else:
        print(f"No data available for year {year}")

    print(f"\nEthnicity Summary for Year: {year}")
    if year in per_year_stats:
        print(per_year_stats[year]['usable_by_ethnicity'].head())
    else:
        print(f"No data available for year {year}")

    for loc_col in ['collection_location_zip', 'collection_location_city', 'collection_location_name']:
        print(f"\nCollection Location {loc_col} for Year: {year}")
        if year in per_year_stats and per_year_stats[year]['pivot_collection'].get(loc_col) is not None:
            print(per_year_stats[year]['pivot_collection'][loc_col].head())
        else:
            print(f"No data available for year {year} or column {loc_col}")

Overall County Summary:
              total_records  usable_records
county_clean                               
Miami-Dade            45552           41669
Broward                7031            6559
Gadsden                 807             805
Palm Beach              654             627
Orange                   92              91

Overall Pivot Zip Sample:
county_clean    Alachua  Bay  Bradford  Brevard  Broward  Charlotte  Clay  \
voting_zipcode                                                              
00003               0.0  0.0       0.0      0.0      0.0        0.0   0.0   
00018               0.0  0.0       0.0      0.0      0.0        0.0   0.0   
00030               0.0  0.0       0.0      0.0      0.0        1.0   0.0   
00032               0.0  0.0       0.0      0.0      0.0        0.0   0.0   
00033               0.0  0.0       0.0      0.0      3.0        0.0   0.0   

county_clean    Collier  Duval  Escambia  ...  Santa Rosa  Sarasota  Seminole  \
voting_zipcode      